## Introduction

This project will focus on comparing the state capitals of every state in the US based on the types of venues in the area.  The capital cities will then be clustered using a support vector machine algorithm.  This algorithm will be used because the clustering will likely not be sphere-like.  There may be sub-groups within larger groups that make sense to set aside as distinct.  There may also be some groups that are similar in many ways, but have some key differences.  SVM will take into account these differences, whereas a different type of algorithm, such as k-means, may not.  The target audience for this project would be people who are looking to expand their businesses from one city to another.  Cities in the same cluster may have like-minded citizens and similar business needs.  For example, if cities A and B are in the same cluster, a business owner in city A may have reason to believe that his or her business would also succeed in city B.  People that are moving to a different city may also have interest in this project.  They could use the clusters to determine which new cities are most like the one where they currently live.

## Data

This project will utilize data from Foursquare to compile the most common types of venues in each of the cities.  It will also use state capital data from a table on a website like Wikipedia and location data accessed with geopy.  For example, the state capital of Maine is found to be Augusta, as shown on this web page https://en.wikipedia.org/wiki/List_of_capitals_in_the_United_States.  This table could be scraped into a pandas dataframe using the BeautifulSoup package.  The coordinates of August are 44.3106, -69.7795, which could be added to the cities dataframe through the use of the geopy python package.  Using the location data in the dataframe, an API call could be made to Foursquare to retrieve the first 200 venues within a 1 mile radius of the given coordinates.  The venues would then be classified using one-hot ecoding of the venue types.  This process would be repeated for the remaining 49 state capitals.  Once this process is finished, the top 10 venue types in each city would be compiled and entered into the SVM algorithm to generate the clusters.